<a href="https://colab.research.google.com/github/pedrocivita/superComp_pedrotpc/blob/main/aula10/supercompAtividade10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Atividade 10 - Supercomputação - Pedro Civita**

In [ ]:
!nvidia-smi

Fri Sep 13 14:31:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## **Exercício 1**

In [8]:
%%writefile exercicio1.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>
#include <chrono>
#include <fstream>

int main() {
    std::ifstream arquivo("stocks-google.txt");
    if (!arquivo.is_open()) {
        std::cerr << "Erro ao abrir o arquivo" << std::endl;
        return -1;
    }

    double preco;
    thrust::host_vector<double> host;  // Vetor na CPU

    // Leitura dos dados de preços do arquivo
    while (arquivo >> preco) {
        host.push_back(preco);
    }

    // Medição do tempo de cópia dos dados para a GPU
    auto start = std::chrono::high_resolution_clock::now();
    thrust::device_vector<double> dev(host);  // Copiando os dados para a GPU
    auto end = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> duration = end - start;
    std::cout << "Tempo de transferência para a GPU: " << duration.count() << " segundos" << std::endl;

    return 0;
}


Writing exercicio1.cu


In [9]:
!nvcc exercicio1.cu -o exercicio1

In [10]:
!./exercicio1

Tempo de transferência para a GPU: 0.106408 segundos


## **Exercício 2**

In [22]:
%%writefile exercicio2.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/reduce.h>
#include <thrust/extrema.h>
#include <iostream>
#include <fstream>
#include <chrono>

int main() {
    // Lendo os dados do arquivo stocks-google.txt
    std::ifstream arquivo("stocks-google.txt");
    if (!arquivo) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    std::vector<double> precos;
    double preco;

    // Lê os preços do arquivo e armazena no vetor precos
    while (arquivo >> preco) {
        precos.push_back(preco);
    }

    arquivo.close();

    // Transfere os dados para o host_vector e depois para o device_vector
    thrust::host_vector<double> host(precos.begin(), precos.end());
    thrust::device_vector<double> dev(host);

    // Cálculo do preço médio dos últimos 10 anos (total)
    int total_dias = dev.size();
    double soma_total = thrust::reduce(dev.begin(), dev.end(), 0.0, thrust::plus<double>());
    double media_total = soma_total / total_dias;
    std::cout << "Preço médio total: " << media_total << std::endl;

    // Cálculo do preço médio dos últimos 365 dias
    int ultimos_365_dias = std::min(365, total_dias);  // Garante que há 365 dias ou menos
    double soma_365 = thrust::reduce(dev.end() - ultimos_365_dias, dev.end(), 0.0, thrust::plus<double>());
    double media_365 = soma_365 / ultimos_365_dias;
    std::cout << "Preço médio nos últimos 365 dias: " << media_365 << std::endl;

    // Encontrar o maior e o menor preço no período total
    auto max_total = thrust::max_element(dev.begin(), dev.end());
    auto min_total = thrust::min_element(dev.begin(), dev.end());
    std::cout << "Maior preço total: " << *max_total << std::endl;
    std::cout << "Menor preço total: " << *min_total << std::endl;

    // Encontrar o maior e o menor preço nos últimos 365 dias
    auto max_365 = thrust::max_element(dev.end() - ultimos_365_dias, dev.end());
    auto min_365 = thrust::min_element(dev.end() - ultimos_365_dias, dev.end());
    std::cout << "Maior preço nos últimos 365 dias: " << *max_365 << std::endl;
    std::cout << "Menor preço nos últimos 365 dias: " << *min_365 << std::endl;

    return 0;
}

Overwriting exercicio2.cu


In [23]:
!nvcc exercicio2.cu -o exercicio2

In [24]:
!./exercicio2

Preço médio total: 1580.08
Preço médio nos últimos 365 dias: 1615.81
Maior preço total: 2200
Menor preço total: 1015.24
Maior preço nos últimos 365 dias: 2200
Menor preço nos últimos 365 dias: 1060.5


## **Exercício 3**

In [25]:
%%writefile exercicio3.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <iostream>
#include <fstream>
#include <sstream>  // Para ler os valores separados por vírgula
#include <vector>   // Para armazenar os preços lidos

int main() {
    // Abrindo o arquivo stocks2.txt
    std::ifstream arquivo("stocks2.txt");
    if (!arquivo) {
        std::cerr << "Erro ao abrir o arquivo stocks2.txt" << std::endl;
        return 1;
    }

    // Vetores para armazenar os preços da Apple e Microsoft
    std::vector<double> precos_apple, precos_microsoft;

    // Variáveis para armazenar os valores lidos
    std::string linha;
    double preco_apple, preco_microsoft;

    // Lê os preços do arquivo e armazena em vetores separados
    while (std::getline(arquivo, linha)) {
        std::stringstream ss(linha);
        std::string valor;

        // Lendo o preço da Apple
        std::getline(ss, valor, ',');
        preco_apple = std::stod(valor);
        precos_apple.push_back(preco_apple);

        // Lendo o preço da Microsoft
        std::getline(ss, valor);
        preco_microsoft = std::stod(valor);
        precos_microsoft.push_back(preco_microsoft);
    }
    arquivo.close();

    // Transferindo os dados para host_vector e depois para device_vector
    thrust::host_vector<double> host_apple(precos_apple.begin(), precos_apple.end());
    thrust::host_vector<double> host_microsoft(precos_microsoft.begin(), precos_microsoft.end());

    thrust::device_vector<double> dev_apple(host_apple);
    thrust::device_vector<double> dev_microsoft(host_microsoft);

    // Vetor para armazenar as diferenças ponto a ponto
    thrust::device_vector<double> diferencas(dev_apple.size());

    // Calcula a diferença ponto a ponto entre os preços da Apple e Microsoft
    thrust::transform(dev_apple.begin(), dev_apple.end(), dev_microsoft.begin(), diferencas.begin(), thrust::minus<double>());

    // Calcula a soma das diferenças
    double soma_diferencas = thrust::reduce(diferencas.begin(), diferencas.end(), 0.0, thrust::plus<double>());

    // Calcula a média das diferenças
    double media_diferencas = soma_diferencas / diferencas.size();

    std::cout << "Média das diferenças (Apple - Microsoft): " << media_diferencas << std::endl;

    return 0;
}


Overwriting exercicio3.cu


In [26]:
!nvcc exercicio3.cu -o exercicio3

In [27]:
!./exercicio3

Média das diferenças (Apple - Microsoft): -265.232


A média negativa indica que, em média, os preços das ações da Microsoft foram mais altos que os da Apple ao longo do período.